# Приложение к расчётно-графическому домашнему заданию

In [1]:
import numpy as np
import pandas as pd

import cvxpy
np.set_printoptions(suppress=True)

# Входные данные

In [2]:
# Входные данные
product_qty = np.array([  # продукты (включая отходы в последней колонке)
    [0.070, 0.888, 0, 0, 0, 0, 0.042],  # технологии
    [0.42, 0, 0, 0, 0, 0.539, 0.041], 
    [0, 0, 0, 0.154, 0, 0.807, 0.039], 
    [0.352, 0, 0.470, 0, 0.158, 0, 0.020], 
    [0, 0, 0.973, 0, 0, 0, 0.027], 
    [0.145, 0, 0, 0, 0.815, 0, 0.040],  
    [0, 0.687, 0, 0, 0, 0.277, 0.036], 
    [0.145, 0, 0, 0, 0, 0.833, 0.022], 
    [0, 0, 0, 0.318, 0.652, 0, 0.03], 
    [0, 0, 0, 0, 0.978, 0, 0.022], 
    [0, 0, 0, 0.795, 0.163, 0, 0.042]
    ])

# производительность (тонн/час)
productivity = np.array([5, 9, 10, 10, 11, 10, 14, 11, 14, 12, 15])
plan = np.array([150, 130, 140, 150, 200, 100])

scheme = [(0, 6), (6, 11)]  # технологическая схема агрегатов

times = [5, 6]  # фонд свободного времени для каждого агрегата

# 1) Построить оптимальный план с первоначальными условиями

In [3]:
def production_plan(product_qty, productivity, plan, unit_scheme, times):
    """Возвращает общее мин. кол-во тонн и расход ресурсов по каждой технологии.
    
    product_qty:  матрица кол-ва продуктов, тонн в форме (технологии X продукты)
    productivity: вектор производительности тонн/час
    plan:         вектор плана, тонн
    unit_scheme:  схема расположения агрегатов в форме [(0, 6), (6, 11)]
    times:        вектор фонда свободного времени по каждому агрегату 
    """
    # кол-во технологий, кол-во продуктов
    tech_num, products_num = product_qty.shape[0], product_qty.shape[1]
    # решающая переменная
    x = cvxpy.Variable(shape=(1, tech_num), integer=False)

    constraints = []  # ограничения по плану, производительности и времени
    # хотим выполнить план - ограничения по плану производства 
    for prod_num in range(products_num):
        constraints.append(x @ product_qty[:, prod_num] >= plan[prod_num])
    # учитываем производительность агрегатов и фонд свободного времени для каждого агрегата 
    for idx, u in enumerate(unit_scheme):
        constraints.append(
            x[:, u[0]:u[1]] @ (1 / productivity[u[0]:u[1]]) <= times[idx]
            )
    # условие неотрицательности
    constraints.append(x >= 0)

    # целевая функция - минимизация совокупного количества (нижний предел - план)
    obj_func = cvxpy.sum(x)
    problem = cvxpy.Problem(cvxpy.Minimize(obj_func), constraints=constraints)
    
    # мин совокупное кол-во и колич-во по каждой технологии
    min_ttl_qty = np.round(problem.solve(solver='ECOS_BB'), 2)
    resource_consumption = x.value

    return min_ttl_qty, resource_consumption

In [4]:
# последний столбец product_qty - величина отходов - не нужен
res = production_plan(product_qty[:, :-1], productivity, plan, scheme, times)
try:
    qty = np.round(res[1], 3)
    print(
    f'Кол-во прод. + отходы (т): {res[0]}\nКол-во прод. по каждой техн. (т):\n {qty}'
    )
except Exception:
    print('Нет решения при таких временных ограничениях')

Нет решения при таких временных ограничениях


# 2) Убрать ограничение по времени и посмотреть, какие минимальные ресурсы нужны на выполнение плана

Произвольно выбираю большое время, чтобы найти минимум ресурсов на выполение плана без учета временых ограничений.

In [5]:
res = production_plan(
    product_qty[:, :-1],
    productivity,
    plan,
    scheme,
    times=[1000, 1000]  # неогран. фонд свободного времени для каждого агрегата
    )

qty = np.round(res[1], 3)
print(
f'Кол-во прод., включая отходы (т): {res[0]}\nКол-во прод. по каждой технологии (т):\n {qty}'
)
print('Кол-во тонн каждого продукта (включая отходы)')
print(np.round(res[1] @ product_qty, 3))  # == np.sum(res[1])

Кол-во прод., включая отходы (т): 896.86
Кол-во прод. по каждой технологии (т):
 [[146.396  53.634   0.    297.872   0.      0.      0.     85.344  86.627
   72.954 154.028]]
Кол-во тонн каждого продукта (включая отходы)
[[150.    130.    140.    150.    200.    100.     26.856]]


In [104]:
# проверка сумм
# np.sum(qty), np.sum(res[1] @ product_qty)

In [6]:
# минимум используемых ресурсов
x_min = res[1]
x_min

array([[146.39638397,  53.63423753,   0.00000127, 297.87234018,
          0.0000002 ,   0.00000889,   0.00001615,  85.34350604,
         86.6269202 ,  72.95375188, 154.02847701]])

# 4) Совокупное минимальное время для выполнения плана

In [106]:
def minimize_total_time(prod_qty, p, plan, min_res=None, tol=None):
    """Возвращает общее мин. время и расход ресурсов по каждой технологии.
    
    prod_qty:  количество тонн j-го продукта, получаемого из 1 тонны ресурса 
               по i-ой технологии 
               размер матрицы (кол-во технологий X кол-во продуктов)
    p:         вектор производительности переработки ресурса по i-ой технологии, тонн в час
    plan:      вектор плана производства j-го продукта, тонны
    min_res:   ограничение по соблюдению минимума расхода ресурсов
               Может быть предоставлен вектор минимума ресурсов
    tol:       допуск на увеличение минимума расходра ресурсов
    """
    # кол-во технологий, кол-во продуктов
    tech_num, products_num = prod_qty.shape[0], prod_qty.shape[1]
    # решающая переменная
    x = cvxpy.Variable(shape=(1, tech_num), integer=False)

    constraints = []
    # хотим выполнить план - ограничения по плану производства 
    for prod_num in range(products_num):
        constraints.append(x @ prod_qty[:, prod_num] >= plan[prod_num])
    # если хотим установить ограничение по расходу ресурсов или допустить 
    #   небольшое увеличение расхода ресурсов по каждой технологии
    if min_res is not None and tol is not None:
        constraints.append(x <= cvxpy.multiply(min_res, tol))
    # условие неотрицательности
    constraints.append(x >= 0)

    # целевая функция - минимизация совокупного времени по двум агрегатам
    # разбивки согласно тех. схеме нет
    obj_func = x @ (1 / p)
    problem = cvxpy.Problem(cvxpy.Minimize(obj_func), constraints=constraints)
    
    # мин общее время и колич-во по каждой технологии
    min_ttl_time = np.round(problem.solve(solver='ECOS_BB'), 2)
    resource_consumption = x.value

    return min_ttl_time, resource_consumption

In [107]:
res = minimize_total_time(
    product_qty[:, :-1],
    productivity,
    plan,
    x_min,
    1      # допуск нулевой, то есть хотим уложиться в минимум расхода ресурсов
    )
print(f'Мин общее время для выполнения плана: {res[0]} часа')
print('Кол-во тонн каждого продукта (включая отходы):')
print(np.round(res[1] @ product_qty, 3))
print(f'Кол-во тонн по каждой технологии:\n {np.round(res[1][0], 3)}')
print('Общий расход ресурсов')
# для проверки -- общий расход ресурсов по технологиям и по продуктам с отходами
np.sum(res[1]), np.sum(res[1] @ product_qty)

Мин общее время для выполнения плана: 95.32 часа
Кол-во тонн каждого продукта (включая отходы):
[[150.    130.    140.    150.    200.    100.     26.856]]
Кол-во тонн по каждой технологии:
 [146.396  53.634   0.    297.872   0.      0.      0.     85.344  86.627
  72.954 154.028]
Общий расход ресурсов


(896.8556430261112, 896.8556430261114)

In [108]:
# отказ от требования по оптимальности потребления ресурсов
res_2 = minimize_total_time(
    product_qty[:, :-1],
    productivity,
    plan,
    # x_min,
    # 1.8      # допуск не влияет, но можно отказаться от ограничения
    )
print(f'Мин общее время для выполнения плана: {res_2[0]}')
print('Кол-во тонн каждого продукта (включая отходы):')
print(np.round(res_2[1] @ product_qty, 3))
print(f'Кол-во тонн по каждой технологии:\n {np.round(res_2[1][0], 3)}')
print('Общий расход ресурсов')
# для проверки -- общий расход ресурсов по технологиям и по продуктам с отходами
np.sum(res_2[1]), np.sum(res_2[1] @ product_qty)

Мин общее время для выполнения плана: 77.03
Кол-во тонн каждого продукта (включая отходы):
[[150.    130.    150.776 150.    200.    100.     27.439]]
Кол-во тонн по каждой технологии:
 [  0.     88.281   0.    320.801   0.      0.    189.229   0.    202.043
   0.    107.862]
Общий расход ресурсов


(908.215531832929, 908.215531832929)

В таблице ниже показано время работы по каждой технологии в сценарии, в котором ищется минимальное время для выполнения плана при соблюдении минимума ресурсов по каждой технологии.

In [109]:
tech_time = []
for time in np.round(res[1]/productivity, 2)[0]:
    tech_time.append(time)
tech_time = np.array(tech_time)
columns = ['Часы']
tech_time_df = pd.DataFrame(data=tech_time, columns=columns)
tech_time_df.index += 1
tech_time_df

,Часы
1,29.28
2,5.96
3,0.00
4,29.79
5,0.00
6,0.00
7,0.00
8,7.76
9,6.19
10,6.08


Время по агрегатам

In [110]:
u_1 = tech_time_df.iloc[0:7, :].sum()[0]
u_2 = tech_time_df.iloc[7:12, :].sum()[0]
print(f'Время по агр. 1 ({u_1} ч.) + по агр. 2 ({u_2} ч.) = {u_1 + u_2} ч')

Время по агр. 1 (65.03 ч.) + по агр. 2 (30.3 ч.) = 95.33 ч


# 5) Отказаться от исходного плана, сохранить ограничения по времени -- Максимизация доходов

Сохраняем время работы агрегатов, убираем план, вводим цену на продукцию. Ищем новый, самый выгодный производственный план. То есть приоритезируем производство самых выгодных продуктов

In [111]:
def maximize_profit(prod_qty, p, prices, unit_scheme, times):
    """Возвращает макс. прибыль и расход ресурсов по каждой технологии.
    
    prod_qty:  количество тонн j-го продукта, получаемого из 1 тонны ресурса 
               по i-ой технологии 
               размер матрицы (кол-во технологий X кол-во продуктов)
    p:         вектор производительности переработки ресурса по i-ой технологии, тонн в час
    prices:    вектор цен на конечную продукцию
    unit_scheme:  схема расположения агрегатов в форме [(0, 6), (6, 11)]
    times:        вектор фонда свободного времени по каждому агрегату
    """
    # кол-во технологий, кол-во продуктов
    tech_num, products_num = prod_qty.shape[0], prod_qty.shape[1]
    # решающая переменная
    x = cvxpy.Variable(shape=(1, tech_num), integer=False)

    constraints = []
    # учитываем производительность агрегатов и фонд свободного времени для каждого агрегата 
    for idx, u in enumerate(unit_scheme):
        constraints.append(
            x[:, u[0]:u[1]] @ (1 / productivity[u[0]:u[1]]) <= times[idx]
            )
    # условие неотрицательности
    constraints.append(x >= 0)

    # целевая функция - минимизация совокупного времени по двум агрегатам
    # разбивки согласно тех. схеме нет
    obj_func = (x @ prod_qty) @ prices 
    problem = cvxpy.Problem(cvxpy.Maximize(obj_func), constraints=constraints)
    
    # мин общее время и колич-во по каждой технологии
    min_ttl_time = np.round(problem.solve(solver='ECOS_BB'), 2)
    resource_consumption = x.value

    return min_ttl_time, resource_consumption

In [112]:
# проверка размерностей
# x = np.ones((1, 11))
# print(x.shape)
# prices = np.array([100, 125, 150, 175, 200, 225])
# print(prices.shape)
# print((x @ product_qty[:, :-1]).shape)
# (x @ product_qty[:, :-1]) @ prices

In [113]:
# определяем цены
prices = np.array([100, 125, 150, 175, 200, 225])
max_prof_res = maximize_profit(
    product_qty[:, :-1],
    productivity,
    prices,
    scheme,
    times
    )

In [114]:
print(f'Максимальный доход {max_prof_res[0]} долл.')
print('Кол-во тонн каждого продукта (и отходы последняя цифра):')
print(np.round(max_prof_res[1] @ product_qty, 3))
print(f'Расход ресурсов по каждой технологии (т):\n {np.round(max_prof_res[1][0], 3)}')
print('Общий расход ресурсов')
# для проверки -- общий расход ресурсов по технологиям и продуктам
np.sum(max_prof_res[1]), np.sum(max_prof_res[1] @ product_qty)

Максимальный доход 26054.45 долл.
Кол-во тонн каждого продукта (и отходы последняя цифра):
[[ 0.     0.     0.    34.412 54.768 40.35   4.47 ]]
Расход ресурсов по каждой технологии (т):
 [ 0.  0. 50.  0.  0.  0.  0.  0. 84.  0.  0.]
Общий расход ресурсов


(134.00000018877688, 134.00000018877688)

In [9]:
# 2) Минимизировать время работы медленного агрегата при введении допуска (1-2%) на увеличение 
# использования ресурсов по сравнению с найденным в п. 1) минимумом используемых ресурсов

In [7]:
def minimize_time_slow_unit(
    x_min, product_qty, productivity, plan, 
    unit_scheme, tolerance
    ):
    """Возвращает мин. время медленного агрегата и расход ресурсов по каждой технологии.
    
    x_min:    минимальный расход ресурсов по каждой технологии
    product_qty:  матрица кол-ва продуктов, тонн в форме (технологии X продукты)
    productivity: вектор производительности тонн/час
    plan:         вектор плана, тонн
    unit_scheme:  схема расположения агрегатов в форме [(0, 6), (6, 11)]
    tolerance:    размер допуска, или уступки, по перерасходу ресурсов
    """
    # кол-во технологий, кол-во продуктов
    u = unit_scheme[0]  # № агрегата
    tech_num, products_num = product_qty.shape[0], product_qty.shape[1]
    # решающая переменная
    x = cvxpy.Variable(shape=(1, tech_num), integer=False)

    constraints = []  # ограничения по плану и расходу ресурсов с допуском 
    # хотим выполнить план - ограничение по плану
    for prod_num in range(products_num):
        constraints.append(x @ product_qty[:, prod_num] >= plan[prod_num])
    # хотим допустить небольшое увеличение расхода ресурсов по каждой технологии
    # constraints.append(x <= cvxpy.multiply(x_min, tolerance))
    # что если делать допуск только по ресурсам по данному агрегату?
    constraints.append(x[:, u[0]:u[1]] <= cvxpy.multiply(x_min[:, u[0]:u[1]], tolerance))
    # условие неотрицательности
    constraints.append(x >= 0)

    # целевая функция - минимизация времени работы медленного агрегата
    u = unit_scheme[0]
    obj_func = cvxpy.sum(x[:, u[0]:u[1]] @ (1 / productivity[u[0]:u[1]]))
    problem = cvxpy.Problem(cvxpy.Minimize(obj_func), constraints=constraints)
    
    # мин время и расход ресурсов по каждой технологии
    min_time = problem.solve(solver='ECOS_BB')
    resource_consumption = x.value

    return min_time, resource_consumption

In [45]:
min_time_slow_unit_res = minimize_time_slow_unit(
    x_min,
    product_qty[:, :-1],
    productivity,
    plan,
    scheme,
    1.02  # допуск или уступка по расходу ресурсов 1.02 = на 2% выше минимума
    )

print(f'Мин. время работы медленного агрегата {min_time_slow_unit_res[0]} часа')

Мин. время работы медленного агрегата 29.7872341111724 часа


In [46]:
min_time_slow_unit_res

(29.7872341111724,
 array([[  0.00000004,   0.00000004,   0.00000044, 297.87234031,
           0.0000001 ,   0.00000014, 232.86746979, 324.92563601,
         123.42285721, 104.00850185, 160.2581152 ]]))

In [47]:
print('Кол-во тонн каждого продукта (включая отходы):')
print(np.round(min_time_slow_unit_res[1] @ product_qty, 3))
print(f'Кол-во тонн по каждой технологии:\n {np.round(min_time_slow_unit_res[1][0], 3)}')
print('Общий расход ресурсов')
# для проверки -- общий расход ресурсов по технологиям и по продуктам с отходами
np.sum(min_time_slow_unit_res[1]), np.sum(min_time_slow_unit_res[1] @ product_qty)

Кол-во тонн каждого продукта (включая отходы):
[[151.965 159.98  140.    166.654 255.378 335.167  34.211]]
Кол-во тонн по каждой технологии:
 [  0.      0.      0.    297.872   0.      0.    232.867 324.926 123.423
 104.009 160.258]
Общий расход ресурсов


(1243.354921138456, 1243.354921138456)

In [48]:
tech_time = []
for time in np.round(min_time_slow_unit_res[1]/productivity, 2)[0]:
    tech_time.append(time)
tech_time = np.array(tech_time)
columns = ['Часы']
tech_time_df = pd.DataFrame(data=tech_time, columns=columns)
# tech_time_df.index += 1
tech_time_df

,Часы
0,0.00
1,0.00
2,0.00
3,29.79
4,0.00
5,0.00
6,16.63
7,29.54
8,8.82
9,8.67


In [49]:
tech_time_df.sum()

Часы    104.13
dtype: float64

In [50]:
u_1 = tech_time_df.iloc[0:6, :].sum()[0]
u_2 = tech_time_df.iloc[6:, :].sum()[0]
print(f'Время по агр. 1 ({u_1} ч.) + по агр. 2 ({u_2} ч.) = {u_1 + u_2} ч')

Время по агр. 1 (29.79 ч.) + по агр. 2 (74.34 ч.) = 104.13 ч


In [32]:
# 3) Минимизировать время работы быстрого агрегата при не увеличении суммарного времени 
# работы обоих агрегатов. !Не забыть при допуск на увеличение ресурсов и здесь
def minimize_time_fast_unit(
    x_min, slow_unit_time, product_qty, productivity, 
    plan, unit_scheme, tolerance
    ):
    """Возвращает мин. время быстрого агрегата и расход ресурсов по каждой технологии.
    
    x_min:    минимальный расход ресурсов по каждой технологии
    slow_unit_time: мин время работы медленного агрегата
    product_qty:  матрица кол-ва продуктов, тонн в форме (технологии X продукты)
    productivity: вектор производительности тонн/час
    plan:         вектор плана, тонн
    unit_scheme:  схема расположения агрегатов в форме [(0, 6), (6, 11)]
    tolerance:    размер допуска, или уступки, по перерасходу ресурсов
    """
    # кол-во технологий, кол-во продуктов
    u = unit_scheme[1]  # № агрегата
    tech_num, products_num = product_qty.shape[0], product_qty.shape[1]
    # решающая переменная
    x = cvxpy.Variable(shape=(1, tech_num), integer=False)

    constraints = []  # ограничения по плану и расходу ресурсов с допуском 
    # хотим выполнить план - ограничение по плану
    for prod_num in range(products_num):
        constraints.append(x @ product_qty[:, prod_num] >= plan[prod_num])
    # хотим допустить небольшое увеличение расхода ресурсов по каждой технологии
    # constraints.append(x <= cvxpy.multiply(x_min, tolerance))
    # что если делать допуск только по ресурсам по данному агрегату?
    constraints.append(x[:, u[0]:u[1]] <= cvxpy.multiply(x_min[:, u[0]:u[1]], tolerance))
    # не хотим превысить суммарное время работы обоих агрегатов
    constraints.append(cvxpy.sum(x @ (1 / productivity)) <= slow_unit_time)
    # условие неотрицательности
    constraints.append(x >= 0)

    # целевая функция - минимизация времени работы быстрого агрегата
    # u = unit_scheme[1]
    obj_func = cvxpy.sum(x[:, u[0]:u[1]] @ (1 / productivity[u[0]:u[1]]))
    problem = cvxpy.Problem(cvxpy.Minimize(obj_func), constraints=constraints)
    
    # мин время и расход ресурсов по каждой технологии
    min_time = problem.solve(solver='ECOS_BB')
    resource_consumption = x.value

    return min_time, resource_consumption

In [39]:
min_time_fast_unit_res = minimize_time_fast_unit(
    min_time_slow_unit_res[1],  # хочу сохранить объем использования первого агрегата вместо  x_min,
    105,
    product_qty[:, :-1],
    productivity,
    plan,
    scheme,
    1.02  # допуск или уступка
    )

print(f'Мин. время работы быстрого агрегата {min_time_fast_unit_res[0]} часа')

Мин. время работы быстрого агрегата 10.40320147242008 часа


In [40]:
print('Кол-во тонн каждого продукта (включая отходы):')
print(np.round(min_time_fast_unit_res[1] @ product_qty, 3))
print(f'Кол-во тонн по каждой технологии:\n {np.round(min_time_fast_unit_res[1][0], 3)}')
print('Общий расход ресурсов')
# для проверки -- общий расход ресурсов по технологиям и по продуктам с отходами
np.sum(min_time_fast_unit_res[1]), np.sum(min_time_fast_unit_res[1] @ product_qty)

Кол-во тонн каждого продукта (включая отходы):
[[150.    130.    157.728 150.    200.    135.942  31.949]]
Кол-во тонн по каждой технологии:
 [146.396   0.    168.453 335.592   0.    149.13    0.      0.      0.
   0.    156.048]
Общий расход ресурсов


(955.6196109026034, 955.6196109026034)

In [41]:
tech_time = []
for time in np.round(min_time_fast_unit_res[1]/productivity, 2)[0]:
    tech_time.append(time)
tech_time = np.array(tech_time)
columns = ['Часы']
tech_time_df = pd.DataFrame(data=tech_time, columns=columns)
# tech_time_df.index += 1
tech_time_df

,Часы
0,29.28
1,0.00
2,16.85
3,33.56
4,0.00
5,14.91
6,0.00
7,0.00
8,0.00
9,0.00


In [38]:
u_1 = tech_time_df.iloc[0:6, :].sum()[0]
u_2 = tech_time_df.iloc[6:, :].sum()[0]
print(f'Время по агр. 1 ({u_1} ч.) + по агр. 2 ({u_2} ч.) = {u_1 + u_2} ч')

Время по агр. 1 (94.6 ч.) + по агр. 2 (10.4 ч.) = 105.0 ч


In [12]:
z = np.ones((1, 11))
z

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [17]:
cvxpy.multiply(x_min, 1.02)

Expression(CONSTANT, NONNEGATIVE, (1, 11))

In [19]:
z <= x_min * 1.02

array([[ True,  True, False,  True, False, False, False,  True,  True,
         True,  True]])

In [11]:
F_12_max, F_12_distr = distribute_two(a)

In [15]:
F_12_max, F_12_distr

([0, 5, 9, 11.5, 13], [[0, 0], [1, 0], [1, 1], [2, 1], [3, 1]])

In [13]:
b = [0, 6, 8, 10, 12]
F_123 = distribute_two([F_12_max, b])
F_123

([0, 6, 11, 15, 17.5], [[0, 0], [0, 1], [1, 1], [2, 1], [3, 1]])

In [14]:
F_123_dist = []
for i in F_123[1]:
    F_123_dist.append(F_12_distr[i[0]] + [i[1]])

F_123_dist

[[0, 0, 0], [0, 0, 1], [1, 0, 1], [1, 1, 1], [2, 1, 1]]

In [16]:
c = [0, 4.5, 7, 9, 11]
F_1234 = distribute_two([F_123[0], c])
F_1234

([0, 6, 11, 15.5, 19.5], [[0, 0], [1, 0], [2, 0], [2, 1], [3, 1]])

In [18]:
F_1234_dist = []
for i in F_1234[1]:
    F_1234_dist.append(F_123_dist[i[0]] + [i[1]])

F_1234_dist

[[0, 0, 0, 0], [0, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 1], [1, 1, 1, 1]]

In [42]:
from pprint import pprint

In [77]:
import numpy as np
import pandas as pd


def distribute_two(a):
    """Return max profit list and distribution list for 2 banks."""
    max_list = []
    opt_list = []

    for i in range(len(a[0])):
        max = a[0][i] + a[1][0]
        opt = [i, 0]
        for j in range(i+1):
            test_max = a[0][i-j] + a[1][j]
            if test_max > max:
                max = test_max
                opt = [i-j, j]
        max_list.append(max)
        opt_list.append(opt)
    return max_list, opt_list


# a = [[0, 5, 7.5, 9, 10],
#      [0, 4, 5.5, 8, 10]
#      ]
# b = [[0, 5, 9, 11.5, 13],
#      [0, 6, 8, 10, 12]
#      ]

# distribute_two(a)
# distribute_two(b)

In [78]:
def distribute(a):
    """Return max profits and distribution of investments.
    
    a : a profit matrix A x B, rows are banks, columns are investments
        a_ij - profit j from bank i 
    """
    a = np.hstack((np.zeros((len(a), 1)), a))
    dist_table = list(a[:2, :])

    F_12_max, F_12_distr = distribute_two(a[0:2, :])
    dist_table.append(F_12_max)
    dist_table.append(F_12_distr)
    for row in a[2:, :]:
        # print(row)
        dist_table.append(row)
        F_123 = distribute_two([dist_table[-3], row])
        F_123_dist = []
        for i in F_123[1]:
            F_123_dist.append(dist_table[-2][i[0]] + [i[1]])
        dist_table.append(F_123[0])    # add maximum profits
        dist_table.append(F_123_dist)  # add optimal distribution

    return dist_table

# a = [[5, 7.5, 9, 10],
#   [4, 5.5, 8, 10],
#   [6, 8, 10, 12],
#   [4.5, 7, 9, 11]
#   ]
# res = distribute(a)
# pprint(res)

In [80]:
def build_column_names(col_num):
    """Return a list of column names."""
    columns = ['f1(x)', 'f2(x)', 'F12(A)', 'F12_dist']
    n = int((col_num - 4) / 3)
    count = 3
    for i in range(n):
        s = f'f{count}(x)'
        columns.append(s)
        r = ''.join([str(i) for i in range(1, count+1)])
        s = f'F{r}(A)'
        columns.append(s)
        s = f'F{r}_dist'
        columns.append(s)
        count += 1
    return columns

In [87]:
# Часть 1 и 2
a = [[5, 7.5, 9, 10],
  [4, 5.5, 8, 10],
  [6, 8, 10, 12],
  [4.5, 7, 9, 11]
  ]
def main(a):
    res = distribute(a)
    dist_table = np.array(res, dtype='object')
    columns = build_column_names(dist_table.shape[0])
    best_dist = pd.DataFrame(dist_table.T, columns=columns)
    return best_dist

best_dist = main(a)
best_dist

,f1(x),f2(x),F12(A),F12_dist,f3(x),F123(A),F123_dist,f4(x),F1234(A),F1234_dist
0,0,0,0,"[0, 0]",0,0,"[0, 0, 0]",0,0,"[0, 0, 0, 0]"
1,5,4,5,"[1, 0]",6,6,"[0, 0, 1]",4.5,6,"[0, 0, 1, 0]"
2,7.5,5.5,9,"[1, 1]",8,11,"[1, 0, 1]",7,11,"[1, 0, 1, 0]"
3,9,8,11.5,"[2, 1]",10,15,"[1, 1, 1]",9,15.5,"[1, 0, 1, 1]"
4,10,10,13,"[3, 1]",12,17.5,"[2, 1, 1]",11,19.5,"[1, 1, 1, 1]"


In [88]:
# часть 3
a = [
  # [5, 7.5, 9, 10],  # убираем 1-ый агрегат
  [4, 5.5, 8, 10],
  [6, 8, 10, 12],
  [4.5, 7, 9, 11]
  ]
best_dist = main(a)
best_dist

,f1(x),f2(x),F12(A),F12_dist,f3(x),F123(A),F123_dist
0,0,0,0,"[0, 0]",0,0,"[0, 0, 0]"
1,4,6,6,"[0, 1]",4.5,6,"[0, 1, 0]"
2,5.5,8,10,"[1, 1]",7,10.5,"[0, 1, 1]"
3,8,10,12,"[1, 2]",9,14.5,"[1, 1, 1]"
4,10,12,14,"[3, 1]",11,17,"[1, 1, 2]"


In [90]:
# пример данных из учебника
b = [[0.28, 0.45, 0.65, 0.78, 0.9, 1.02, 1.13, 1.23, 1.32],
     [0.25, 0.41, 0.55, 0.65, 0.75, 0.8, 0.85, 0.88, 0.9],
     [0.15, 0.25, 0.4, 0.5, 0.62, 0.73, 0.82, 0.9, 0.96]
     ]

best_dist = main(b)
best_dist

,f1(x),f2(x),F12(A),F12_dist,f3(x),F123(A),F123_dist
0,0,0,0,"[0, 0]",0,0,"[0, 0, 0]"
1,0.28,0.25,0.28,"[1, 0]",0.15,0.28,"[1, 0, 0]"
2,0.45,0.41,0.53,"[1, 1]",0.25,0.53,"[1, 1, 0]"
3,0.65,0.55,0.7,"[2, 1]",0.4,0.7,"[2, 1, 0]"
4,0.78,0.65,0.9,"[3, 1]",0.5,0.9,"[3, 1, 0]"
5,0.9,0.75,1.06,"[3, 2]",0.62,1.06,"[3, 2, 0]"
6,1.02,0.8,1.2,"[3, 3]",0.73,1.21,"[3, 2, 1]"
7,1.13,0.85,1.33,"[4, 3]",0.82,1.35,"[3, 3, 1]"
8,1.23,0.88,1.45,"[5, 3]",0.9,1.48,"[4, 3, 1]"
9,1.32,0.9,1.57,"[6, 3]",0.96,1.6,"[5, 3, 1]"
